In [1]:
import xarray as xr
import pandas as pd
data_temp = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Temp\*.nc", parallel=False)
data_tmax = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_tmax\*.nc", parallel=False)
data_tmin = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_tmin\*.nc", parallel=False)
data_rain = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Rain\*.nc", parallel=False)
data_hurs = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Humidity\*.nc", parallel=False)
data_sun = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Sun\*.nc", parallel=False)
data_frost = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Frost\*.nc", parallel=False)
data_psl = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_psl\*.nc", parallel=False)
data_wind = xr.open_mfdataset(r"C:\Users\iarla\OneDrive\Documents\MSc_Project\HadUK_data\12km_Month_Wind\*.nc", parallel=False)

In [2]:
import numpy as np

rain = np.array(data_rain['rainfall'])
hurs = np.array(data_hurs['hurs'])
temp = np.array(data_temp['tas'])
temp_max = np.array(data_tmax['tasmax'])
temp_min = np.array(data_tmin['tasmin'])
sun = np.array(data_sun['sun'])
frost = np.array(data_frost['groundfrost'])
psl = np.array(data_psl['psl'])
wind = np.array(data_wind['sfcWind'])

min_length = len(wind)
rain = rain[:min_length]
hurs = hurs[:min_length]
temp = temp[:min_length]
temp_max = temp_max[:min_length]
temp_min = temp_min[:min_length]
sun = sun[:min_length]
frost = frost[:min_length]
wind = wind[:min_length]
psl = psl[:min_length]

In [3]:
import torch 

land_mask = ~np.isnan(temp)
land_mask_torch = torch.from_numpy(land_mask.astype(bool)) 

print(temp.shape)
print(land_mask.shape)

(648, 112, 82)
(648, 112, 82)


In [4]:
from skimage import measure

# Connected components labeling
labels = measure.label(land_mask)
biggest_component_label = np.argmax(np.bincount(labels.flat)[1:]) + 1  

# Isolate the biggest component
main_landmass_mask = (labels == biggest_component_label) 

# Get indices for bounding box
nonzero_indices = main_landmass_mask.nonzero()
min_row = np.min(nonzero_indices[1])
max_row = np.max(nonzero_indices[1])
min_col = np.min(nonzero_indices[2])
max_col = np.max(nonzero_indices[2])

In [5]:
rain = rain[:, min_row:max_row + 1, min_col:max_col + 1]
hurs = hurs[:, min_row:max_row + 1, min_col:max_col + 1]
temp = temp[:, min_row:max_row + 1, min_col:max_col + 1]
temp_max = temp_max[:, min_row:max_row + 1, min_col:max_col + 1]
temp_min = temp_min[:, min_row:max_row + 1, min_col:max_col + 1]
sun = sun[:, min_row:max_row + 1, min_col:max_col + 1] 
frost = frost[:, min_row:max_row + 1, min_col:max_col + 1] 
psl = psl[:, min_row:max_row + 1, min_col:max_col + 1] 
wind = wind[:, min_row:max_row + 1, min_col:max_col + 1] 
land_mask = land_mask[:, min_row:max_row + 1, min_col:max_col + 1]

In [6]:
nan_counts = np.count_nonzero(np.isnan(rain), axis=0)  # Counts across time

valid_grid_cell_mask = nan_counts == 0
valid_cell_indices = np.where(valid_grid_cell_mask)

# Your arrays from the previous output:
row_indices = valid_cell_indices[0]
col_indices = valid_cell_indices[1]

unique_rows, row_counts = np.unique(row_indices, return_counts=True)
unique_cols, col_counts = np.unique(col_indices, return_counts=True)

In [7]:
valid_row_indices = valid_cell_indices[0]  # Extract row indices
valid_col_indices = valid_cell_indices[1]  # Extract column indices

valid_rain_data = rain[:, valid_row_indices, valid_col_indices]
valid_temp_data = temp[:, valid_row_indices, valid_col_indices]
valid_wind_data = wind[:, valid_row_indices, valid_col_indices]
valid_hurs_data = hurs[:, valid_row_indices, valid_col_indices]
valid_psl_data = psl[:, valid_row_indices, valid_col_indices]
valid_frost_data = frost[:, valid_row_indices, valid_col_indices]
valid_sun_data = sun[:, valid_row_indices, valid_col_indices]
valid_tmax_data = temp_max[:, valid_row_indices, valid_col_indices]
valid_tmin_data = temp_min[:, valid_row_indices, valid_col_indices]

In [8]:
valid_rain_data = valid_rain_data[:,:1600].reshape(648, 64, 25)
valid_temp_data = valid_temp_data[:,:1600].reshape(648, 64, 25)
valid_wind_data = valid_wind_data[:,:1600].reshape(648, 64, 25)
valid_hurs_data = valid_hurs_data[:,:1600].reshape(648, 64, 25)
valid_psl_data = valid_psl_data[:,:1600].reshape(648, 64, 25)
valid_sun_data = valid_sun_data[:,:1600].reshape(648, 64, 25)
valid_frost_data = valid_frost_data[:,:1600].reshape(648, 64, 25)
valid_tmax_data = valid_tmax_data[:,:1600].reshape(648, 64, 25)
valid_tmin_data = valid_tmin_data[:,:1600].reshape(648, 64, 25)

In [9]:
nan_count_temp = np.isnan(valid_temp_data).sum()
nan_count_tmax = np.isnan(valid_tmax_data).sum()
nan_count_tmin = np.isnan(valid_tmin_data).sum()
nan_count_wind = np.isnan(valid_wind_data).sum()

print('Number of NaN values in Temp:', nan_count_temp)
print('Number of NaN values in Tmax:', nan_count_tmax)
print('Number of NaN values in Tmin:', nan_count_tmin)
print('Number of NaN values in Wind:', nan_count_wind)


def fill_nan_with_mean(data):
    col_mean = np.nanmean(data, axis=0)  # Calculate mean per column
    inds = np.where(np.isnan(data))
    data[inds] = np.take(col_mean, inds[1])  # Replace NaNs with mean
    return data

temp_filled = fill_nan_with_mean(valid_temp_data.copy())  
tmax_filled = fill_nan_with_mean(valid_tmax_data.copy())
tmin_filled = fill_nan_with_mean(valid_tmin_data.copy())
wind_filled = fill_nan_with_mean(valid_wind_data.copy())

nan_count_temp = np.isnan(temp_filled).sum()
nan_count_tmax = np.isnan(tmax_filled).sum()
nan_count_tmin = np.isnan(tmin_filled).sum()
nan_count_wind = np.isnan(wind_filled).sum()

print('Number of NaN values in Temp:', nan_count_temp)
print('Number of NaN values in Tmax:', nan_count_tmax)
print('Number of NaN values in Tmin:', nan_count_tmin)
print('Number of NaN values in Tmin:', nan_count_tmin)

Number of NaN values in Temp: 123
Number of NaN values in Tmax: 108
Number of NaN values in Tmin: 123
Number of NaN values in Wind: 167
Number of NaN values in Temp: 0
Number of NaN values in Tmax: 0
Number of NaN values in Tmin: 0
Number of NaN values in Tmin: 0


In [10]:
rain_tensor = torch.from_numpy(valid_rain_data)
temp_tensor = torch.from_numpy(temp_filled)
hurs_tensor = torch.from_numpy(valid_hurs_data)
frost_tensor = torch.from_numpy(valid_frost_data)
sun_tensor = torch.from_numpy(valid_sun_data)
wind_tensor = torch.from_numpy(wind_filled)
psl_tensor = torch.from_numpy(valid_psl_data)
tmax_tensor = torch.from_numpy(tmax_filled)
tmin_tensor = torch.from_numpy(tmin_filled)

In [11]:
variables = [rain_tensor, temp_tensor, hurs_tensor, frost_tensor, sun_tensor, wind_tensor, psl_tensor, tmax_tensor, tmin_tensor] 

# Stack along the channels dimension (dim=1)
stacked_tensor = torch.stack(variables, dim=1) 

stacked_array = stacked_tensor.numpy() 
print('Stacked array shape:', stacked_array.shape)

Stacked array shape: (648, 9, 64, 25)


In [ ]:
import torch.nn as nn
import torch


class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, bias=True):
        super(ConvLSTMCell, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(
            in_channels=input_dim + hidden_dim,  # Sum of input and hidden state dimensions
            out_channels=4 * hidden_dim,
            kernel_size=self.kernel_size,
            padding=self.padding,
            bias=self.bias
        )

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state
        combined = torch.cat([input_tensor, h_cur], dim=1)
        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)
        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))


class ConvLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

        self.final_conv = nn.Conv2d(in_channels=self.hidden_dim[-1], out_channels=1, 
                                    kernel_size=(1, 1), padding=0, bias=bias)

    def forward(self, input_tensor, hidden_state=None):
        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b, image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)

        for layer_idx in range(self.num_layers):
            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=input_tensor[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])


        final_output_sequence = []
        for t in range(seq_len):
            # Extract the output for each time step across all batches
            timestep_output = layer_output_list[-1][:, t, :, :, :]  # Select the output of the last layer
            # Apply the final convolutional layer
            timestep_output_reduced = self.final_conv(timestep_output)
            final_output_sequence.append(timestep_output_reduced.unsqueeze(1))

        # Concatenate the time steps to form the final output sequence
        final_output_sequence = torch.cat(final_output_sequence, dim=1)

        return final_output_sequence 

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param


In [ ]:
from convlstm import ConvLSTM
import torch  

seq_length = 12
num_layers = 1  
hidden_dim = [64]
kernel_size = [(3,3)]  
input_channels = 9   

model = ConvLSTM(input_dim=input_channels,
                    hidden_dim=hidden_dim,
                    kernel_size=kernel_size,
                    num_layers=num_layers,
                    batch_first=True,  
                    return_all_layers=False)

In [ ]:
def create_input_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequence = data[i:i+seq_length]  # Shape: (seq_length, 9, 64, 25)
        sequences.append(sequence)
    return np.array(sequences)

def create_target_sequences(data, seq_length, forecast_length):
    targets = []
    for i in range(len(data) - seq_length - forecast_length + 1):
        # Adjust this line to gather a sequence of 'forecast_length' future frames as the target
        target = data[i + seq_length:i + seq_length + forecast_length, 1, :, :]
        targets.append(target)
    return np.array(targets)

seq_length = 12  # This represents how many timesteps back the model should look
forecast_length = 12
# Assuming stacked_array is your input data with shape (648, 9, 64, 25)
input_sequences = create_input_sequences(stacked_array, seq_length)
target_sequences = create_target_sequences(stacked_array, seq_length, forecast_length)

input_sequences = input_sequences[:625]

# Check the shapes to ensure everything looks right
print("Input sequences shape:", input_sequences.shape)
print("Target sequences shape:", target_sequences.shape)

In [ ]:
import torch

# Convert numpy arrays to PyTorch tensors
input_sequences_tensor = torch.tensor(input_sequences, dtype=torch.float)
target_sequences_tensor = torch.tensor(target_sequences, dtype=torch.float)

# Add a channel dimension to the target tensor to match the expected input shape of the model
target_sequences_tensor = target_sequences_tensor.unsqueeze(2)  

print("Input tensor shape:", input_sequences_tensor.shape)
print("Target tensor shape:", target_sequences_tensor.shape)

In [ ]:
from sklearn.model_selection import train_test_split

inputs_temp, inputs_test, targets_temp, targets_test = train_test_split(
    input_sequences_tensor, target_sequences_tensor, test_size=0.2, random_state=42
)

inputs_train, inputs_val, targets_train, targets_val = train_test_split(
    inputs_temp, targets_temp, test_size=0.25, random_state=42  
)

print("Training set shapes:", inputs_train.shape, targets_train.shape)
print("Validation set shapes:", inputs_val.shape, targets_val.shape)
print("Test set shapes:", inputs_test.shape, targets_test.shape)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Create TensorDatasets
train_data = TensorDataset(inputs_train, targets_train)
val_data = TensorDataset(inputs_val, targets_val)
test_data = TensorDataset(inputs_test, targets_test)
print(f"Prepared Train Dataset Size: {len(train_data)} samples")
print(f"Prepared Validation Dataset Size: {len(val_data)} samples")
print(f"Prepared Test Dataset Size: {len(test_data)} samples")
# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
print(f"Train DataLoader Length: {len(train_loader)} batches")
print(f"Validation DataLoader Length: {len(val_loader)} batches")
print(f"Test DataLoader Length: {len(test_loader)} batches")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import r2_score, mean_absolute_error

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define number of epochs
num_epochs = 300

# Lists to store training and validation losses
train_losses = []
val_losses = []

# Lists to store R-squared values and MAE
train_r2_scores = []
val_r2_scores = []
train_maes = []
val_maes = []

# Training loop
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        
        optimizer.zero_grad()
        outputs = model(inputs)

        
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * inputs.size(0)
        
    predictions_train = []
    targets_train = []
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        
        # Collect predictions and targets for calculating R-squared and MAE
        predictions_train.append(outputs.detach().numpy())
        targets_train.append(targets.numpy())
    
    # Calculate R-squared and MAE for training set
    predictions_train = np.concatenate(predictions_train)
    predictions_train = predictions_train.reshape(predictions_train.shape[0], -1)
    targets_train = np.concatenate(targets_train)
    targets_train = targets_train.reshape(targets_train.shape[0], -1)
    train_r2 = r2_score(targets_train, predictions_train)
    train_mae = mean_absolute_error(targets_train, predictions_train)
    
    # Calculate average training loss
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)
    train_r2_scores.append(train_r2)
    train_maes.append(train_mae)
    
    # Validation
    model.eval()
    val_loss = 0.0
    predictions_val = []
    targets_val = []
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)
            
            # Collect predictions and targets for calculating R-squared and MAE
            predictions_val.append(outputs.detach().numpy())
            targets_val.append(targets.numpy())
    
    # Calculate R-squared and MAE for validation set
    predictions_val = np.concatenate(predictions_val)
    predictions_val = predictions_val.reshape(predictions_val.shape[0], -1)
    targets_val = np.concatenate(targets_val)
    targets_val = targets_val.reshape(targets_val.shape[0], -1)
    val_r2 = r2_score(targets_val, predictions_val)
    val_mae = mean_absolute_error(targets_val, predictions_val)
    
    # Calculate average validation loss
    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)
    val_r2_scores.append(val_r2)
    val_maes.append(val_mae)
    
    # Print training/validation statistics
    print(f'Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train R2: {train_r2:.4f}, Val R2: {val_r2:.4f}, Train MAE: {train_mae:.4f}, Val MAE: {val_mae:.4f}')

# Plotting loss and metrics
import matplotlib.pyplot as plt

# Loss
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# R-squared
plt.plot(train_r2_scores, label='Train R2')
plt.plot(val_r2_scores, label='Val R2')
plt.xlabel('Epoch')
plt.ylabel('R-squared')
plt.title('Training and Validation R-squared')
plt.legend()
plt.show()

# MAE
plt.plot(train_maes, label='Train MAE')
plt.plot(val_maes, label='Val MAE')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Training and Validation Mean Absolute Error')
plt.legend()
plt.show()

In [ ]:
model.eval()

with torch.no_grad():
    predictions_test = []
    targets_test = []

    for inputs, targets in test_loader:

        # Generate predictions
        outputs = model(inputs)

        # Store predictions and targets for evaluation
        predictions_test.append(outputs.cpu().numpy())
        targets_test.append(targets.cpu().numpy())

# Concatenate all batches
predictions_test = np.concatenate(predictions_test, axis=0)
targets_test = np.concatenate(targets_test, axis=0)

# Flatten the arrays to make them compatible with scikit-learn metrics
predictions_test_flat = predictions_test.reshape(-1)
targets_test_flat = targets_test.reshape(-1)

# Calculate R² and MAE
test_r2 = r2_score(targets_test_flat, predictions_test_flat)
test_mae = mean_absolute_error(targets_test_flat, predictions_test_flat)

print(f'Test R²: {test_r2:.4f}')
print(f'Test MAE: {test_mae:.4f}')